In [2]:
import pandas as pd
import os
from itertools import permutations
from sklearn.preprocessing import MinMaxScaler
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Gather 2022 Details 
data_dir = 'C:/Users/harry/Documents/data/Project_Node/2022-08-14-171850/'
person = pd.read_csv(os.path.join(data_dir, 'PersonHistorical.csv'))

# Filter Person ID by Internal Netherland Office people 
filtered_person_df = person[(person['Country']=='Netherlands') & (person['WorkerType']=='Employee') & \
                            (person['Domain']=='CUSHWAKE.COM') & (person['ActiveStatus']=='Yes') & \
                            (person['IsInternal']==True) ]

# Helix Office
# filtered_person_df = filtered_person_df[filtered_person_df['Office'].isin(['C - NLD - Utrecht - Franz-Lisztplantsoen 100', 
#                                                       'C - NLD - Naarden - Gooimeer 6-32'])]

In [4]:
# Gather 2021 Details
data_dir_2 = 'C:/Users/harry/Documents/data/Project_Node/2021/'
person_2021 = pd.read_csv(os.path.join(data_dir_2, 'PersonHistorical.csv'))

# Filter Person ID by Internal Netherland Office people 
filtered_person_2021 = person_2021[(person_2021['Country']=='Netherlands') & (person_2021['WorkerType']=='Employee') & \
                            (person_2021['Domain']=='CUSHWAKE.COM') & (person_2021['ActiveStatus']=='Yes') & \
                            (person_2021['IsInternal']==True) ]

# Helix Office
# filtered_person_2021 = filtered_person_2021[filtered_person_2021['Office'].isin(['C - NLD - Utrecht - Franz-Lisztplantsoen 100', 
#                                                       'C - NLD - Naarden - Gooimeer 6-32'])]

filtered_person_2021.sort_values(by='StartDate', ascending=False, inplace=True)
filtered_person_2021.drop_duplicates(subset='EmailAddress', inplace=True, keep='first')

In [32]:
grandmaster_df = pd.DataFrame()
for comm_type in ['Calls', 'connectivity', 'IMS', 'Mails', 'Meetings']:
    data_2021_dir = 'C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output/'
    file_path = os.path.join(data_2021_dir, '*{}*all_nodes.xlsx'.format(comm_type))
    file_list_2021 = glob(file_path)
    print('2021', file_list_2021[0])
    file_df_2021 = pd.read_excel(file_list_2021[0])
    file_df_2021.rename({'EmailAdress': 'EmailAddress'}, inplace=True, axis=1)
    file_df_2021.columns = ['Unnamed: 0',
     'EmailAddress',
     'Eigenvector_centrality_2021',
     'Betweeness_centrality_2021',
     'Closeness_centrality_2021',
     'Degree_centrality_2021']

    data_2022_dir = 'C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2022/Output/'
    file_path = os.path.join(data_2022_dir, '*{}*all_nodes.xlsx'.format(comm_type))
    file_list_2022 = glob(file_path)
    file_df_2022 = pd.read_excel(file_list_2022[0])
    print('2021', file_list_2022[0])
    file_df_2022.rename({'EmailAdress': 'EmailAddress'}, inplace=True, axis=1)
    file_df_2022.columns = ['Unnamed: 0',
     'EmailAddress',
     'Eigenvector_centrality_2022',
     'Betweeness_centrality_2022',
     'Closeness_centrality_2022',
     'Degree_centrality_2022']

    master_df =  file_df_2021.merge(file_df_2022, on='EmailAddress', how='inner')
    master_df = master_df.merge(filtered_person_2021, on='EmailAddress', how='inner')
    master_df.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=True)
    master_df['CommType'] = comm_type
    grandmaster_df = pd.concat([grandmaster_df, master_df])
    
grandmaster_df.loc[grandmaster_df['CommType']=='connectivity', 'CommType'] = 'Overall'

2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output\Callsall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2022/Output\Callsall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output\connectivity_scoreall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2022/Output\connectivity_scoreall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output\IMSall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2022/Output\IMSall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output\Mailsall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2022/Output\Mailsall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output\Meetingsall_data_all_nodes.xlsx
2021 C:/Users/harry/Documents/data/Proj

In [33]:
file_path = os.path.join(data_2022_dir, '*all_data.xlsx')
file_list_2022 = glob(file_path)
file_df_2022 = pd.read_excel(file_list_2022[0])

file_path = os.path.join(data_2021_dir, '*all_data.xlsx')
file_list_2021 = glob(file_path)
file_df_2021 = pd.read_excel(file_list_2021[0])

df = file_df_2022[['Unnamed: 0', 'mean']].merge(file_df_2021[['Unnamed: 0', 'mean']], on='Unnamed: 0')
df.columns = ['Centrality', '2022 Average', '2021 Average']
df['Avg_change_in_percentage'] = (df['2022 Average'] - df['2021 Average']) / df['2021 Average']

df['Centrality'] = df['Centrality'].str.replace('connectivity_scoreall', 'Overallall')
df['CommType'] = df['Centrality'].str.split('_', expand=True)[0].str[:-3]
df['CentralityType'] = df['Centrality'].str.split('_', expand=True)[2]


In [35]:
# Re-create Overall Scores 

df = df[df['CommType']!='Overall']
a  = df.groupby(by='CentralityType').mean().reset_index()
a['CommType'] = 'Overall'
a['Centrality'] = a['CommType'] + '_data_' + a['CentralityType'] + '_centrality'
df = pd.concat([df, a])

grandmaster_df = grandmaster_df[grandmaster_df['CommType']!='Overall']
a = grandmaster_df.groupby('EmailAddress').mean().reset_index()
a['CommType'] = 'Overall'

b = grandmaster_df[grandmaster_df['CommType']=='Calls']
b_cols = [item for item in b.columns if item not in a.columns] + ['EmailAddress']
b = b[b_cols]
a = a.merge(b, on='EmailAddress')

grandmaster_df = pd.concat([grandmaster_df, a])

In [36]:
report_dir = 'C:/Users/harry/Documents/data/Project_Node/'
file_path = os.path.join(data_2022_dir, '*all_nodes.xlsx')
file_list_2022 = glob(file_path)

with pd.ExcelWriter(os.path.join(report_dir, 'Netherlands Study Scenario 2.xlsx')) as writer:  
    grandmaster_df.to_excel(writer, sheet_name='All Nodes S2', index=False)
    df.to_excel(writer, sheet_name='Centralities Stats S2', index=False)

In [37]:
for file in file_list_2022:
    file_df = pd.read_excel(file)
    file_df.rename({'EmailAdress': 'EmailAddress'}, inplace=True, axis=1)
    file_df2 = file_df.merge(filtered_person_2021, on='EmailAddress', how='inner')
    print(file_df.shape, file_df2.shape)

(428, 6) (428, 37)
(370, 6) (370, 37)
(428, 6) (428, 37)
(390, 6) (390, 37)
(417, 6) (417, 37)
(411, 6) (411, 37)


In [42]:
for file in file_list_2021:
    file_df = pd.read_excel(file)
    file_df.rename({'EmailAdress': 'EmailAddress'}, inplace=True, axis=1)
    print(file_df.shape)

(20, 9)


In [43]:
file_list_2021

['C:/Users/harry/Documents/data/Project_Node/Helix_Analysis_2021/Output\\Characteristics_avg_all_communication_all_data.xlsx']